In [10]:
import pandas as pd
import numpy as np
from numpy import diff

from scipy.stats import ttest_ind

import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import ternary # ternary plot has a bug
import plotly.express as px
import h5py
import matplotlib as mpl

# import geopandas as gpd
import math

from matplotlib.patches import Circle, Wedge, Polygon

import os
import matplotlib.patheffects as path_effects


from netCDF4 import Dataset as NetCDFFile 

# for beachball - there are some error with using obspy
from pyrocko.plot import beachball
from pyrocko import moment_tensor as ptm

import cartopy.crs as ccrs
import cartopy.mpl.geoaxes

from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import matplotlib.ticker as mticker
from pathlib import Path
%matplotlib inline

import harmonica as hm

from obspy.core import UTCDateTime

from datetime import datetime, timezone

import seaborn as sns
sns.set()
sns.set_style(style='white') 
sns.set_style("ticks", {"xtick.major.size": 0.5, "ytick.major.size": 0.5})


#set the font globally
plt.rcParams.update({'font.family':'Helvetica'})

In [11]:
# data = pd.read_csv('sum_with_2005_eqs.csv', header = 'infer')
data = pd.read_csv("/Users/mohd0001/Research/PhD_Monash/Paper2/time-series/data/ryu_kur_class_by_loc_net.csv", header = 'infer')
# data.dtypes

data.sort_values(by = 'MW', ascending = False, inplace = True)

data['DATE'] = pd.to_datetime(data['DATE']) # change dtype to datetime format
data['DATE'] = data['DATE'].dt.strftime('%Y-%m-%d') # change dtype to object

# data['DATE'] and data['TIME'] should be object (str) to perform + operator

data_dummy = np.array(data['DATE'] + ' ' + data['TIME'])

data_ev_time = []

for val in data_dummy:
    data_ev_time.append(UTCDateTime(val))

    
ref_date = data_ev_time[0]

data_ev_time = np.array(data_ev_time)

date = []

for val in data_ev_time:
    date.append(val - ref_date)
    
date = np.array(date) # this is in seconds

data['YEARS'] = date/31536000

data = data.sort_values(by ='YEARS', ascending=True) # this is an important step
# data = data[(data['DATE'] > '01-01-2003') & (data['DATE'] < '26-11-2004')]

In [12]:
data_mthrust = data[(data['CLASS'] == 'MTHRUST') &
                    (data['MW'] >= 7.5) &
                    (data['LAT'] > 34) & (data['LAT'] < 55)]


data_DW = data[(data['CLASS'] == 'DGOING') &
               (data['DEPTH'] > 20) & (data['DEPTH'] < 225) &
               (data['MW'] >= 6) &
              (data['LAT'] < 34) & (data['LAT'] > 23.5) &
              (data['LON'] > 123)]

In [13]:
## olive cluster (-35, -32)

t1 = -35
t2 = -32

data_dg_olive = data_DW[(data_DW['YEARS'] > t1) & (data_DW['YEARS'] < t2)]

# print('Olive band')
print('Long slab')
print(data_dg_olive[['LAT','LON','DEPTH', 'YEARS', 'MW', 'DATE']])


data_mt_olive = data_mthrust[(data_mthrust['YEARS'] > t1) & (data_mthrust['YEARS'] < t2)]

print('Megathrust')
print(data_mt_olive[['LAT','LON','DEPTH', 'YEARS', 'MW', 'DATE']])

Long slab
        LAT      LON  DEPTH      YEARS   MW        DATE
421  28.298  130.637   23.1 -34.259094  6.4  1976-12-14
409  31.097  130.158  164.1 -32.821682  6.7  1978-05-23
Megathrust
         LAT      LON  DEPTH      YEARS   MW        DATE
1380  45.085  148.257   25.0 -32.989328  7.6  1978-03-23
1381  44.243  148.916   12.3 -32.984700  7.5  1978-03-24
1379  38.242  142.021   52.4 -32.766841  7.6  1978-06-12


In [14]:
## orange band

t1 = -18
t2 = -16

data_dg_orange = data_DW[(data_DW['YEARS'] > t1) & (data_DW['YEARS'] < t2)]

# print('orange band')
print('Long slab')
print(data_dg_orange[['LAT','LON','DEPTH', 'YEARS', 'MW', 'DATE']])


data_mt_orange = data_mthrust[(data_mthrust['YEARS'] > t1) & (data_mthrust['YEARS'] < t2)]

print('Megathrust')
print(data_mt_orange[['LAT','LON','DEPTH', 'YEARS', 'MW', 'DATE']])

Long slab
        LAT      LON  DEPTH      YEARS   MW        DATE
418  26.621  125.665  162.9 -17.603397  6.4  1993-08-07
Megathrust
         LAT      LON  DEPTH      YEARS   MW        DATE
1371  43.842  147.340   35.0 -16.442967  8.3  1994-10-04
1377  40.540  143.448   28.9 -16.210211  7.7  1994-12-28


In [15]:
## maroon band

t1 = -12
t2 = -7

data_dg_maroon = data_DW[(data_DW['YEARS'] > t1) & (data_DW['YEARS'] < t2)]

# print('maroon band')
print('Long slab')
print(data_dg_maroon[['LAT','LON','DEPTH', 'YEARS', 'MW', 'DATE']])


data_mt_maroon = data_mthrust[(data_mthrust['YEARS'] > t1) & (data_mthrust['YEARS'] < t2)]

print('Megathrust')
print(data_mt_maroon[['LAT','LON','DEPTH', 'YEARS', 'MW', 'DATE']])

Long slab
        LAT      LON  DEPTH      YEARS   MW        DATE
444  26.062  124.572  198.0 -11.122045  6.0  2000-01-28
Megathrust
         LAT      LON  DEPTH     YEARS   MW        DATE
1372  41.866  143.861   27.0 -7.461408  8.3  2003-09-25


In [16]:
## purple band

t1 = -6
t2 = 1

data_dg_purple = data_DW[(data_DW['YEARS'] > t1) & (data_DW['YEARS'] < t2)]

# print('purple band')
print('Long slab')
print(data_dg_purple[['LAT','LON','DEPTH', 'YEARS', 'MW', 'DATE']])


data_mt_purple = data_mthrust[(data_mthrust['YEARS'] > t1) & (data_mthrust['YEARS'] < t2)]

print('Megathrust')
print(data_mt_purple[['LAT','LON','DEPTH', 'YEARS', 'MW', 'DATE']])

Long slab
        LAT      LON  DEPTH     YEARS   MW        DATE
416  25.311  123.390  194.5 -5.404329  6.4  2005-10-15
428  31.012  130.125  149.7 -5.302987  6.2  2005-11-21
413  23.580  123.592   29.4 -1.565031  6.6  2009-08-17
431  31.204  130.124  164.9 -1.516933  6.2  2009-09-03
407  25.955  128.525   22.5 -1.033932  7.0  2010-02-26
426  24.299  125.249   47.7 -0.431997  6.3  2010-10-04
443  30.033  131.810   26.1  0.080273  6.0  2011-04-09
Megathrust
         LAT      LON  DEPTH     YEARS   MW        DATE
1373  46.589  153.285   15.0 -4.319924  8.3  2006-11-15
1370  38.228  142.671   14.0  0.000000  9.0  2011-03-11
1374  36.209  141.115   29.0  0.000056  7.9  2011-03-11


In [19]:
## yellow band

t1 = -7
t2 = -4

data_dg_yellow = data_DW[(data_DW['YEARS'] > t1) & (data_DW['YEARS'] < t2)]

# print('yellow band')
print('Long slab')
print(data_dg_yellow[['LAT','LON','DEPTH', 'YEARS', 'MW', 'DATE']])


data_mt_yellow = data_mthrust[(data_mthrust['YEARS'] > t1) & (data_mthrust['YEARS'] < t2)]

print('Megathrust')
print(data_mt_yellow[['LAT','LON','DEPTH', 'YEARS', 'MW', 'DATE']])

Long slab
        LAT      LON  DEPTH     YEARS   MW        DATE
447  26.523  128.882   33.9 -6.637902  6.0  2004-07-22
405  33.096  136.701   23.6 -6.514572  7.2  2004-09-05
416  25.311  123.390  194.5 -5.404329  6.4  2005-10-15
428  31.012  130.125  149.7 -5.302987  6.2  2005-11-21
Megathrust
         LAT      LON  DEPTH     YEARS   MW        DATE
1373  46.589  153.285   15.0 -4.319924  8.3  2006-11-15
